In [101]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import os
import matplotlib.pyplot as plt
import time

In [124]:
coin_labels = os.listdir("Data/Train")

In [122]:
def load_and_process(folder):
    images = []
    labels = []
    for label in coin_labels:
        path = os.path.join(folder, label)
        for filename in os.listdir(path):
            image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            image = cv2.medianBlur(image, 5)
            circles = cv2.HoughCircles(
                        image,
                        cv2.HOUGH_GRADIENT,
                        dp=1,
                        minDist=20,
                        param1=100,
                        param2=30,
                        minRadius=100,
                        maxRadius=1000
                    )
            if circles is not None:
                circles = np.uint16(np.around(circles))
                x, y, r = circles[0, 0]
                size = r * 2
                x_start = max(0, x-r)
                y_start = max(0, y-r)
                roi = image[y_start:y_start+size, x_start:x_start+size]
                if (roi.shape[0] > 0) and roi.shape[1] > 0:
                    roi = cv2.resize(roi, (256,256))
                    roi = cv2.equalizeHist(roi)
                    images.append(roi)
                    labels.append(label)
                    # plt.imshow(roi, cmap='gray')
                    
            else:
                print("Warning! circles were not found")
                image = cv2.resize(image, (256,256))
                image = cv2.equalizeHist(image)
                images.append(image)
                labels.append(label)
                # plt.imshow(image, cmap='grey')
            # plt.show()

    return images, labels
    

In [ ]:
# def load_and_augment(folder):
#     images = []
#     labels = []
#     angles = [90, 180, 270]
#     for label in coin_labels:
#         path = os.path.join(folder, label)
#         print(path)
#         files = os.listdir(path)
#         print(files)
#         for filename in os.listdir(path):
#             image = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
#         if image is not None:
#             for angle in angles:
#                 rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE * angle // 90)
#                 filename = os.path.join(path, str(len(os.listdir(path))) + ".png")
#                 # print(filename)
#                 # cv2.imwrite(filename, rotated)
                
        

In [ ]:
# loading distinct datasets for training and testing
train_images, train_labels = load_and_process("/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/Train")
test_images, test_labels = load_and_process("/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/Test")

# print(f"Train dataset size: {len(train_images)}. Labels: {set(train_labels)}")
# print(f"Test dataset size: {len(test_images)}. Labels: {set(test_labels)}")


/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain/5_rubles
['14.png', '3.png', '13.png', '12.png', '6.png', '4.png', '0.png', '5.png', '16.png', '11.png', '17.png', '7.png', '9.png', '2.png', '1.png', '15.png', '10.png', '8.png', '18.png']
/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain/5_kopecks
['14.png', '3.png', '13.png', '12.png', '6.png', '4.png', '0.png', '5.png', '16.png', '11.png', '17.png', '7.png', '9.png', '2.png', '1.png', '15.png', '10.png', '8.png', '18.png']
/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain/10_kopecks
['14.png', '3.png', '13.png', '12.png', '6.png', '4.png', '0.png', '5.png', '11.png', '7.png', '9.png', '2.png', '1.png', '10.png', '8.png']
/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain/1_ruble
['3.png', '6.png', '4.png', '0.png', '7.png', '2.png', '1.png']
/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain/50_kopecks
['14.png', '3.png', '13.png', '12.png', '6.png', '4.png', '0.png', '[\'3.png\', \'6.

In [ ]:

train_folder = "/home/kolya/code/study/cv/cvlabs/Lab7surf/Data/ClearedTrain"
for label, image in zip(train_labels, train_images):
    pathname = os.path.join(train_folder, label)
    filename = os.path.join(pathname, str(os.listdir(pathname).__len__()))
    print(filename)
    cv2.imwrite((filename + ".png"), image)

# test_folder = "Data/ClearedTest"
# for label, image in zip(test_labels, test_images):
#     pathname = os.path.join(test_folder, label)
#     cv2.imwrite(os.path.join(pathname, str(os.listdir(pathname).__len__()) + ".png"), image)
    

In [ ]:
# creating surf descriptors and adding them together
surf = cv2.xfeatures2d.SURF_create(hessianThreshold=200, extended=True)
all_descs = []
for image in train_images:
    _, descriptor = surf.detectAndCompute(image, None)
    print("keypoints: ", len(_))
    all_descs.append(descriptor)
all_descs = np.vstack(all_descs)
print(f"All descriptors size: {all_descs.shape}")

In [ ]:
# create vocabulary of descriptors using K-means clusterization
N_CLUSTERS = 1500
descs_clusters = KMeans(n_clusters=N_CLUSTERS, random_state=42).fit(all_descs)

In [76]:
# feature vector is a hitogram of cluserized vocabulary descriptors matches
train_features = []
for image in train_images:
    _, descriptor = surf.detectAndCompute(image, None)
    histogram = np.zeros(N_CLUSTERS)
    labels = descs_clusters.predict(descriptor)
    for label in labels:
        histogram[label] += 1
    train_features.append(histogram / (np.sum(histogram)) + 1e-6) # normalize each histogram
train_features = np.array(train_features)

In [77]:
# create and train SVM classifier with train features and labels
svm = SVC(kernel='rbf', C=5.0, gamma='scale')
svm.fit(train_features, train_labels)

SVC(C=5.0)

In [78]:
# test feature vectors are computed the same way
test_features = []
for image in test_images:
    _, descriptor = surf.detectAndCompute(image, None)
    histogram = np.zeros(N_CLUSTERS)
    labels = descs_clusters.predict(descriptor)
    for label in labels:
        histogram[label] += 1
    test_features.append(histogram / (np.sum(histogram)) + 1e-6) # normalize each histogram
test_features = np.array(test_features)

In [79]:
# predict our results
predictions = svm.predict(test_features)
accuracy = np.mean(predictions == test_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 37.21%
